# Setup

In [1]:
import ee
import rasterio
import numpy as np
import pandas as pd
import geopandas as gpd
import tensorflow as tf
from shapely.geometry import Point
from rasterio.plot import show
import matplotlib.pyplot as plt

from naip_cnn.config import CRS

2023-08-03 22:57:54.300109: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-03 22:57:54.420353: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-08-03 22:57:54.931440: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-08-03 22:57:54.931516: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinf

In [ ]:
ee.Authenticate()
ee.Initialize()

# Functions

### 1. sample_LIDAR

function to return a feature collection and a data frame with n samples of data from the named file. Both objects contain (1) sample pixel IDs, (2) row and col IDs, (3) x and y coordinates, and (4) the lidar data value at that pixel. 

In [ ]:
def sample_LIDAR(fname, n):
    # Read the raster and sample from it
    # -------------------------------------------------------------------------#
    with rasterio.open(fname) as src:
        lidar_rast = src.read(1, masked=True)
        profile = src.profile
        transform = src.transform

        fig = plt.figure(figsize=(8, 8))
        show(src, cmap="jet", title=fname.split("/")[-1].split(".")[0])

        lidar_rast_flat = lidar_rast.flatten()
        plt.hist(lidar_rast_flat, bins=100)
        plt.title("Distribution of values in the map")
        plt.show()

    # Get an index of pixels which are not masked
    validPix = np.ma.where(np.ma.getmaskarray(lidar_rast_flat) == False)[0]

    # Sample the not-masked pixel IDs
    sample_IDs = np.random.choice(validPix, size=n, replace=False)

    # Pull out the data at those sample points
    lidar_data_sample = lidar_rast_flat[sample_IDs]
    # -------------------------------------------------------------------------#

    # Show the distribution of sample values
    plt.hist(lidar_data_sample, bins=100)
    plt.title(f"Distribution of values in the sample (n = {n:,})")
    plt.show()

    # Make sample point information
    # -------------------------------------------------------------------------#
    # Get row and column positions of the sample_IDs
    row_id, col_id = np.indices(lidar_rast.shape)
    row_id = row_id.flatten()
    col_id = col_id.flatten()
    row_samp = row_id[sample_IDs]
    col_samp = col_id[sample_IDs]

    # Get upper left coordinates and pixel size
    ul_x = profile["transform"][2]
    ul_y = profile["transform"][5]
    pix_size_x = profile["transform"][0]
    pix_size_y = profile["transform"][4]

    # Calculate x and y coordinates
    x_coords = [(ul_x + (col_i * pix_size_x) + pix_size_x / 2) for col_i in col_samp]
    y_coords = [(ul_y + (row_i * pix_size_y) + pix_size_y / 2) for row_i in row_samp]

    # sample_df = pd.DataFrame({'ID': sample_IDs,'row':row_samp,'col':col_samp,'x_coords':x_coords,'y_coords':y_coords, 'lidar_data': lidar_data_sample,'lidar_raster_ID': fname.split('/')[-1].split('.')[0]})

    # Show the first five rows
    # print(f'First five rows of the sample dataframe \n{sample_df.loc[:5]}')
    # -------------------------------------------------------------------------#

    # Make geodataframe
    # ------------------------------------------------------------------------#
    # Create an empty GeoDataFrame to store the features
    gdf = gpd.GeoDataFrame()

    # Iterate over the centroids and create Point geometries
    for i in range(n):
        point_i = gpd.GeoDataFrame(geometry=[Point(x_coords[i], y_coords[i])])
        gdf = pd.concat([gdf, point_i], ignore_index=True)

    gdf.crs = CRS
    gdf.transform = transform
    gdf["pix_id"] = sample_IDs
    gdf["row"] = row_samp
    gdf["col"] = col_samp
    gdf["lidar_data"] = lidar_data_sample
    gdf["lidar_raster_ID"] = fname.split("/")[-1].split(".")[0]

    # Print the first five rows
    print(f"First five rows of the sample GeoDataFrame \n{gdf.loc[:5]}")
    # ------------------------------------------------------------------------#

    # Define rectangler function
    # ------------------------------------------------------------------------#
    def rectangler(f, buffer_size):
        # Extract centroid coordinates from the feature
        centroid_x, centroid_y = f.geometry.centroid.x, f.geometry.centroid.y

        # Calculate rectangle coordinates
        min_x = centroid_x - buffer_size
        min_y = centroid_y - buffer_size
        max_x = centroid_x + buffer_size
        max_y = centroid_y + buffer_size

        # Create the rectangle geometry using the coordinates
        rectangle = ee.Geometry.Rectangle(
            coords=[min_x, min_y, max_x, max_y],
            proj=CRS,
            geodesic=False,
        )

        # Create an Earth Engine feature
        feature = ee.Feature(None)
        feature = feature.setGeometry(rectangle)
        feature = feature.set("pix_id", f["pix_id"])
        feature = feature.set("row", f["row"])
        feature = feature.set("col", f["col"])
        feature = feature.set("lidar_data", f["lidar_data"])
        feature = feature.set("lidar_raster_ID", f["lidar_raster_ID"])

        return feature

    # ------------------------------------------------------------------------#

    # Make feature collection of rectangles to sample from
    # ------------------------------------------------------------------------#
    # Example usage
    buffer_size = 15.0  # Buffer size in coordinate units

    # Convert the GeoDataFrame to a list of features
    features = gdf.apply(lambda row: rectangler(row, buffer_size), axis=1).tolist()

    # Create an Earth Engine feature collection
    feature_collection = ee.FeatureCollection(features)
    # ------------------------------------------------------------------------#

    # Return results
    return feature_collection, gdf

### 2. sample_NAIP

In [ ]:
def sample_NAIP(featureCollection, GDF, yr):
    # Region to display and clip to
    map_region = ee.Geometry.Polygon(
        [
            [
                [-120.5141935519472, 43.58307281277678],
                [-117.88022138397845, 43.58307281277678],
                [-117.88022138397845, 44.84291584106584],
                [-120.5141935519472, 44.84291584106584],
            ]
        ],
        None,
        False,
    )

    # Defining the LIDAR transform, subbing in NAIP pixel sizes (1 and -1) for the landsat pixel sizes (30 and -30)
    lidar_transform_1m = list(GDF.transform[0:6])
    lidar_transform_1m[0] = 1
    lidar_transform_1m[4] = -1

    # The NAIP data
    image = (
        ee.ImageCollection("USDA/NAIP/DOQQ")
        .filterDate(f"{yr}-01-01", f"{yr}-12-31")
        .median()
        .clip(map_region)
        .reproject(crs=CRS, crsTransform=lidar_transform_1m)
    )

    # Define a function for reducing the NAIP data within a rectangle to a dictionary of values
    # --------------------------------------------------------------------------------------------#
    def reduce_region(feature):
        reduced_data = image.reproject(CRS, lidar_transform_1m).reduceRegion(
            reducer=ee.Reducer.toList(), geometry=feature.geometry(), scale=1
        )
        data_dict = ee.Dictionary(reduced_data)
        feature = feature.set(data_dict)
        return feature

    # --------------------------------------------------------------------------------------------#

    # Bring the data local and return it
    # --------------------------------------------------------------------------------------------#
    reduced_data = featureCollection.map(reduce_region)
    data_dict = reduced_data.getInfo()
    # --------------------------------------------------------------------------------------------#
    return data_dict

### 3. dict_to_TFrecord 

In [ ]:
def dict_to_TFrecord(data_dict, tfrecord_path):
    # Get the features out of the data dictionary
    feature_list = data_dict["features"]

    # Empty lists to store image data and labels in
    dat = []
    labels = []

    # Loop through the features and build the imagery array (X) and the data labels
    # -------------------------------------------------------------------------------------#
    for i in range(len(feature_list)):
        # Pull out the properties of this feature and check which image bands are present
        properties_i = feature_list[i]["properties"]
        properties_i_keys = properties_i.keys()
        bands = [key for key in ["B", "G", "N", "R"] if key in properties_i_keys]

        # Make an image dictionary out of the present bands
        image_dict = {band: properties_i[band] for band in bands}

        # Checking to make sure there isn't empty data in here
        if properties_i["B"] is None:
            continue
        if properties_i["lidar_data"] is None:
            continue

        # Make the image data and lable objects
        dat.append(
            np.array(
                [
                    np.array(image_dict[bands[j]]).reshape(30, 30)
                    for j in range(0, len(image_dict))
                ],
                dtype=np.float32,
            )
        )
        labels.append(feature_list[i]["properties"]["lidar_data"])

    X = np.swapaxes(np.stack(dat), 1, 3).astype("float32")
    labels = np.array(labels, dtype=np.float32)
    # -------------------------------------------------------------------------------------#

    # Define a function that creates an example object for tensorflow
    # ----------------------------------------------------------------------------------#
    def create_example_object_for_tf(image, label):
        feature = {
            "image": tf.train.Feature(bytes_list=tf.train.BytesList(value=[image])),
            "label": tf.train.Feature(bytes_list=tf.train.BytesList(value=[label])),
        }
        example_proto = tf.train.Example(features=tf.train.Features(feature=feature))
        return example_proto.SerializeToString()

    # ----------------------------------------------------------------------------------#

    # Write the training data to a TFrecord
    # ----------------------------------------------------------------------------------#
    # Open a file connection for the TFRecordWriter
    writer = tf.io.TFRecordWriter(tfrecord_path)

    for i in range(len(labels)):
        label = labels[i].tobytes()
        image = X[i, :, :, :].tobytes()

        example_proto = create_example_object_for_tf(image, label)
        writer.write(example_proto)

    # Close the file connection!
    writer.close()
    # ----------------------------------------------------------------------------------#

    print("TFrecord written!")

# Run the sampling

### 1. Set options

In [ ]:
# LIDAR file to sample from
fname = "/scratch/LIDAR/TIFS/MAL2010_FIRST_RETURNS_all_cover_above1p5_30METERS.tif"

# Number of samples to draw
n = 5000

# Year of NAIP data to pull
naip_yr = 2011

# Where to write the training data
tfrecord_path = "/scratch/CNN/training_data/NAIP_x_LIDAR_training_data_TEST.TFrecord"

### 2. Sample from the LIDAR

In [ ]:
sampled_FC, sampled_GDF = sample_LIDAR(fname, n)

### 3. Sample from the NAIP

The NAIP data lives on the GEE server. The sample_NAIP function is going to be pulling data down and may take a while to do so. 

In [ ]:
sampled_DICT = sample_NAIP(sampled_FC, sampled_GDF, naip_yr)

### 4. Store training data in a TFrecord 

In [ ]:
dict_to_TFrecord(sampled_DICT, tfrecord_path)

# Optional

### 1. See the NAIP imagery for the year we're using

In [ ]:
# Set the malheur map region
map_region = ee.Geometry.Polygon(
    [
        [
            [-120.5141935519472, 43.58307281277678],
            [-117.88022138397845, 43.58307281277678],
            [-117.88022138397845, 44.84291584106584],
            [-120.5141935519472, 44.84291584106584],
        ]
    ],
    None,
    False,
)

# The NAIP data
image = (
    ee.ImageCollection("USDA/NAIP/DOQQ")
    .filterDate(f"{naip_yr}-01-01", f"{naip_yr}-12-31")
    .median()
    .clip(map_region)
)


# Use folium to visualize the NAIP data
# --------------------------------------------------------------------------------------------#
mapid = image.getMapId({"bands": ["R", "G", "B"], "min": 0, "max": 255})
map = folium.Map(location=[44.62157, -118.98257])
folium.TileLayer(
    tiles=mapid["tile_fetcher"].url_format,
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay=True,
    name="median composite",
).add_to(map)
map
# --------------------------------------------------------------------------------------------#

### 2. See six randomly drawn sample image / label pairs from the training data 

In [ ]:
i_list = [24000, 130, 24020, 29870, 120, 1321]

fig, axs = plt.subplots(nrows=2, ncols=3, figsize=(36, 24))
axs_flat = axs.flatten()

for i in range(0, len(i_list)):
    ax = axs_flat[i]
    samp = i_list[i]
    ax.imshow(np.flip(X[samp, :, :, 0:3], 2))
    ax.set_title(f"canopy cover: {canCov[samp][0]} %", fontsize=24)
plt.show()